## Load the Teacher

Here, we load the teacher model we trained and we distill it into a student.
For completeness, we also create a regular non distilled student here. We do this three times - once in the main file, once here, and lastly in Student3Distillation.ipynb file.

In [1]:
!pip install keras.utils

  Preparing metadata (setup.py) ... done
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2631 sha256=65b35892a14d0f2a90a2243bfd37b5b5e13aef5e4a93a2361ad5a3860b820c04
  Stored in directory: /root/.cache/pip/wheels/5c/c0/b3/0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils


In [2]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [3]:
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [4]:
import os
import keras
from keras import layers
from keras import ops
import numpy as np
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import pathlib

from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import OneHotEncoder

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [6]:
batch_size = 64
num_classes = 10
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255
test_images = test_images / 255
num_classes = 10
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_images = np.reshape(train_images, (-1, 32, 32, 3))
test_images = np.reshape(test_images, (-1, 32, 32, 3))

In [7]:
teacher = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Dropout(0.3),

        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Dropout(0.5),

        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Dropout(0.5),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),

        layers.Dropout(0.5),

        layers.Dense(num_classes),
    ],
    name="teacher",
)


In [8]:
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [12]:
teacher.load_weights("/teacher.weights.h5")

In [13]:
#we verify that its loaded well
teacher.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.4044 - sparse_categorical_accuracy: 0.8701


[0.4072171747684479, 0.868399977684021]

In [15]:
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(8, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(8, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),


        layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(num_classes),
    ],
    name="student",
)

student_scratch = keras.models.clone_model(student)

#student.summary()

In [ ]:
class Distiller_new(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.2,
        temperature=3,
    ):
        """Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def compute_loss(
        self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False
    ):
        teacher_pred = self.teacher(x, training=False)
        student_loss = self.student_loss_fn(y, y_pred)

        distillation_loss = self.distillation_loss_fn(
            ops.softmax(teacher_pred / self.temperature, axis=1),
            ops.softmax(y_pred / self.temperature, axis=1),
        ) * (self.temperature**2)

        loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
        return loss

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self.student(x, training=True)

            loss = self.compute_loss(x, y, y_pred)

        gradients = tape.gradient(loss, self.student.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.student.trainable_variables))

        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        y_pred = self.student(x, training=False)

        loss = self.compute_loss(x, y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
# Initialize and compile distiller
d = Distiller_new(student= student, teacher=teacher)
d.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.2,
    temperature=5,
)

# Train and evaluate student trained from scratch.
checkpoint_path = "training_student_distilled1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


# Distill teacher to student with validation data
dis = d.fit(train_images, train_labels , epochs=50, batch_size= 64)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:578: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


782/782 ━━━━━━━━━━━━━━━━━━━━ 208s 257ms/step - sparse_categorical_accuracy: 0.4380 - loss: 0.0148
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 256ms/step - sparse_categorical_accuracy: 0.6453 - loss: 0.0592
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 203s 259ms/step - sparse_categorical_accuracy: 0.6958 - loss: 0.0679
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 258s 254ms/step - sparse_categorical_accuracy: 0.7312 - loss: 0.0652
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 204s 256ms/step - sparse_categorical_accuracy: 0.7494 - loss: 0.0637
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 202s 256ms/step - sparse_categorical_accuracy: 0.7633 - loss: 0.0636
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 200s 254ms/step - sparse_categorical_accuracy: 0.7794 - loss: 0.0599
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 204s 257ms/step - sparse_categorical_accuracy: 0.7830 - loss: 0.0606
Epoch 9/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 200s 255ms/step - sparse_categorical_accuracy: 0.7865 - loss: 0.0596
Epoch 10/50
782/782 ━━━━━━━━━━

In [ ]:
d.save_weights("/content/distilled_student_2.weights.h5")

In [ ]:
d.evaluate(test_images,test_labels)

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:578: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - sparse_categorical_accuracy: 0.7911 - loss: 0.0411


[0.04546638950705528,
 0.04546638950705528,
 0.7900000214576721,
 0.7900000214576721]

In [ ]:
# Train student as done usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
checkpoint_path = "training_student_scratch1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


scratch = student_scratch.fit(train_images, train_labels, batch_size=64, epochs=50)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 1.7356 - sparse_categorical_accuracy: 0.3898
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 1.1184 - sparse_categorical_accuracy: 0.6059
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.9288 - sparse_categorical_accuracy: 0.6751
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.8308 - sparse_categorical_accuracy: 0.7074
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.7561 - sparse_categorical_accuracy: 0.7359
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.7102 - sparse_categorical_accuracy: 0.7511
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6637 - sparse_categorical_accuracy: 0.7663
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.6190 - sparse_categorical_accuracy: 0.7828
Epoch 9/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5835 - sparse_categorical_accuracy: 0.7940
Epoch 10/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step -

In [ ]:
student_scratch.save_weights("/content/nondistilled_student_2.weights.h5")

In [ ]:
student_scratch.evaluate(test_images,test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.4310 - sparse_categorical_accuracy: 0.7156


[1.4460475444793701, 0.713100016117096]